The requirements are:
google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread oauth2client, re, numpy

Please do:

pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread  numpy

to install them

In [8]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib gspread oauth2client numpy


In [13]:
from lib.youtube_handler import *
from lib.gsheet_manager import *
import time
import random
from datetime import datetime

client_secrets_file = "my_client.json"  # Path to your client secrets file
youtube_manager = YouTubeManager(client_secrets_file)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=195848401684-rshqm3h42ijd18qvk9n619snq9nvo7tg.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58033%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=MwNzUqCjC7jGNkP3ll0AFCtR0EZWex&access_type=offline


In [10]:
from lib.gsheet_manager import *
service_account_file = "sheet_client.json"
playlist_id = "PLP641Bf8QxbVdSd3qQGSePIx1HChpmfmQ"  # Replace with your actual playlist ID
debug = SheetDebugManager(service_account_file)
requests = SheetRequestManager(service_account_file)
queue = SheetQueueManager(service_account_file)

In [11]:
youtube_manager.remove_all_videos_from_playlist( playlist_id)

All videos have been removed from the playlist.


In [22]:
youtude_interplay = ["https://www.youtube.com/watch?v=owu16UvLqGo",
                     "https://www.youtube.com/watch?v=n8IiMSXqIAo",
                     "https://www.youtube.com/watch?v=lNgH9b4xq0c" ]
user_last_request = {}
while True:
    auth_users    = SheetAuthorizedUsersManager(service_account_file).get_usernames()
    auth_singers  = SheetAuthorizedUsersManager(service_account_file).get_singers()
    users2singers = dict(list(zip(auth_users,auth_singers)))

    records = requests.read_records();
    if len(records)>0:
        for record in requests.read_records():
            #Validate user
            current_user = str(record['Usuario']).lower()
            if current_user in auth_users: #We validate proper user

                if current_user not in user_last_request:
                        user_last_request[current_user] = '5/02/2022 22:35:00'
                # Convert the string to a datetime object
                last_request_time = (datetime.strptime(user_last_request[current_user], '%d/%m/%Y %H:%M:%S')).timestamp()
                new_request_time = (datetime.strptime(record["Marca temporal"], '%d/%m/%Y %H:%M:%S')).timestamp()
                diff_time = new_request_time - last_request_time
                user_last_request[current_user] = record["Marca temporal"]
                if( diff_time < 60 ):
                    debug.add_record(list(record.values())+["Cooldown error: "+str(diff_time)+" seconds"])
                else:
                    current_link = record['Link de Youtube']
                    if youtube_manager.is_valid_youtube_link(current_link):
                        send_youtube =True                        
                        queue_record = [record["Marca temporal"],
                                        users2singers[current_user],
                                        record['Autor/Anime'],
                                        record['Titulo'],record['Link de Youtube']]
                        print(queue_record)
                        try:
                            queue.add_record(queue_record)
                        except:
                            debug.add_record(list(record.values())+["Unkown problem adding song to the queue"])
                            send_youtube =False                        
                        if send_youtube:
                            choice= random.randint(0,2)
                            youtube_manager.add_video_to_playlist(youtude_interplay[choice], playlist_id)
                            youtube_manager.add_video_to_playlist(current_link, playlist_id)                    
                    else:
                        debug.add_record(list(record.values())+["Not a valid youtube link"])            
            else:
                debug.add_record(list(record.values())+["Not a valid user"])
    requests.delete_all_records()
    time.sleep(10) # Sleep for 3 seconds


63396072.0
9.0


In [ ]:
youtube_manager.add_video_to_playlist(youtude_interplay[0], playlist_id)
                        

HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?part=snippet&alt=json returned "Playlist should use manual sorting to support position.". Details: "[{'message': 'Playlist should use manual sorting to support position.', 'domain': 'youtube.playlistItem', 'reason': 'manualSortRequired'}]">